# Dashboard: Student Status Engine                            

In [1]:
#for large datasets, use dask.dataframe
import pandas as pd 
import numpy as np
import json
from pandas.io.json import json_normalize
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display, clear_output


In [2]:
#raw_file = Path.cwd() / "label_predictions.json"
raw_file = "label_predictions.json"

with open(raw_file) as f:
    data = json.load(f)

# converting json dataset from dictionary to dataframe
original_df = pd.DataFrame.from_dict(data)
#df = pd.DataFrame.from_dict(json_normalize(data))
#original_df.head()

In [3]:
pd.set_option('mode.chained_assignment', None)

# Creating new dataframe with just the columns needed
df = original_df[['predictedUser','courseId','revisedLabel','revisedConfidence','predictBy','createdDate','predictedDate','modifiedDate']]

# Prediction dates for instructors are NaN. Filtering predictions by MACH.
df_MACH = df[df['predictedDate'].notna()]

# Extracting date values from dictionary
df_MACH['predictedDate'] = [dates.get('$date') for dates in df_MACH.predictedDate]

# Convert epoch time to human-readable date and time 
#df_MACH['predictedDate'] = df_MACH.predictedDate.map(lambda x: datetime.fromtimestamp(x/1000).strftime("%Y-%m-%d"))
df_MACH['predictedDate'] = pd.to_datetime(df_MACH['predictedDate'],unit='ms')
#df_MACH.head()

In [4]:
# Bin the dates by weeks and compute the weekly predcitions made by the machine
# The first week starts from 07-06-2020

predictions_MACH_week = df_MACH.predictedDate.groupby(df_MACH.predictedDate.dt.week).count()
predictions_MACH_month = df_MACH.predictedDate.groupby(df_MACH.predictedDate.dt.month).count()
#predictions_MACH_month.plot(kind='bar',title='Predictions per month')
#predictions_MACH_week.plot(kind='bar',title='Predictions per week')
# df_MACH[df_MACH.predictedDate==pd.Timestamp(2020,6,9)].count()

In [5]:
# mydata = {'Week': predictions_MACH_week,
#           'Month': predictions_MACH_month}
          
# default = 'Week'

# # Use this to show the numbers when you hover over the plot
# # fig = px.bar(predictions_MACH_week)
# # fig.show()

# mydata[default].plot(kind='bar')

# w = widgets.Dropdown(
#     options=mydata.keys(),
#     value = default,
#     description='Predictions',
# )

# display(w)

# def on_change(change):
#     if change['name'] == 'value' and (change['new'] != change['old']):
#         clear_output()
#         display(w)
#         mydata[change['new']].plot(kind='bar')

# w.observe(on_change)

# # w = widgets.interactive(mydata[default].plot(kind='bar'),w)
# # display(w)

In [6]:
# # Prediction dates for instructors are NaN. Filtering predictions by INS.
# df_INS = df[df['predictedDate'].isnull()]

# print('The total number of MACH predictions in Student Status Engine: {}'.format(predictions_MACH_week.sum()))
# print('The total number of INS predictions in Student Status Engine: {}'.format(len(df_INS)))
# print('Use the dropdown to see a finer details')

In [9]:
#Prediction dates for instructors are NaN. Filtering predictions by INS.
df_INS = df[df['predictedDate'].isnull()]

fig1 = go.Figure(go.Pie(
    values = [predictions_MACH_week.sum(), len(df_INS)],
    labels = ["Predictions", "Annotations"],
    title_text="Predictions and Annotations for SSE",
    texttemplate = "%{label}: %{value:$,s} <br>(%{percent})",
    textposition = "inside"))

fig1.show()

In [13]:
mydata = {'Week': predictions_MACH_week,
          'Month': predictions_MACH_month}

fig = go.FigureWidget()
keys = mydata.keys()

@widgets.interact(Predictions=keys)
def update_px(Predictions):
    p = px.bar(mydata[Predictions])
    for i in range(len(p.data)):
        fig.data = []
    fig.update(data = [d.to_plotly_json() for d in p.data])
    fig.update_layout( height=600,title_text='Predictions by the Student Status Engine')
    fig.plotly_relayout(p.layout.to_plotly_json())
    
fig

interactive(children=(Dropdown(description='Predictions', options=('Week', 'Month'), value='Week'), Output()),…

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'variable=predictedDate<…